In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import similaritymetrics as ims
import datetime
import math

lambdaa = 80
lambdas = 200
taus = 200
taua = 200

def frameDevFunction(i, j, v):
    frameCostValue = min(math.pow(((j-i)-v),2), taus)
    return frameCostValue

def frameAccelCost(h, i, j):
    frameaccelerationcost = min(math.pow(((j-i) - (h-i)),2), taua)
    return frameaccelerationcost

def plotError(frameCount, extraDiag):
    print "Now plotting errors"
    #Create our array of zeros as a 2D numpy array
    errors = np.zeros((frameCount, frameCount))
    #Now Populate our 2 Diagonal Errors
    #Current structure allows image0 loaded once and reused
    for i in range(0, frameCount):
        image1 = cv2.imread("frame%0*d.png" %(4,i))
        image2 = cv2.imread("frame%0*d.png" %(4,i+1))
        #First Diagonal is y=x+1
        errors[i, i+1]=ims.dssimCompare(image1, image2)
        #Second Diagonal is y=x+2
        #This loop calls our extra diagonals for however many we specify
        #We need to add guarding here to prevent looking for an image that extends beyond available frames
        for j in range(0, extraDiag):
            image2 = cv2.imread("frame%0*d.png" %(4,i+3+j))
            #null pointer type exception
            if(image2 is None):
                continue
            errors[i, i+2+j]=ims.dssimCompare(image1, image2)    
    #Save our array
    np.savetxt('array.txt', errors)
    print "Array saved to disk"
    #Now plot and we need a way to save our plot
    plt.imshow(errors, cmap='hot', interpolation='nearest')
    plt.show()
    plt.savefig("errorplot.png")
    print "Plot Saved to Disk"
    
def plotErrorDict(frameCount, extraDiag):
    print "Now Plotting Errors"
    errors = np.zeros((frameCount, frameCount))
    #Create Dictionary with initial items
    imagedict = {}
    for i in range(0, frameCount):
        #Add Entries to Dictionary, only add a single entry when i>1
        if(i>0):
            imagedict["{0}".format(i+extraDiag+1)]= cv2.imread("frame%0*d.png" %(4,i+extraDiag+1))
        else:
            for j in range(i, extraDiag+2):   
                imagedict["{0}".format(j)]= cv2.imread("frame%0*d.png" %(4,j))
        #Then Compare
        for k in range(i+1, i+extraDiag+2):
            if(imagedict["%d" %(k)] is None):
                continue
            errors[i,k] = ims.mse(imagedict["%d" %(i)],imagedict["%d" %(k)])
        #Then delete as needed, we'll delete the earliest created entry
        del imagedict["%d" %(i)]
    np.savetxt('array.txt', errors)
    print "Array saved to disk"
    #Now plot and we need a way to save our plot
    plt.imshow(errors, cmap='hot', interpolation='nearest')
    plt.show()
    plt.savefig("errorplot.png")
    print "Plot Saved to Disk"
    
    
def arrayPath(frameCount, extraDiag, v):
    #V is targetted speed up; v<=extraDiag
    print "Loading Array"
    errors = np.loadtxt('array.txt')
    print "Array Loaded"
    #Add our speed up cost function
    for i in range(0, frameCount):
        for j in range(i+1, i+extraDiag+2):
            if(j<=frameCount & i<=frameCount):
                errors[i,j] += lambdas * frameDevFunction(i, j, v)
    print "Speed Up Cost Added"
    #Array is populated now assess costs
    
    costmatrix = np.zeros((frameCount, frameCount))
    #First Pass of Populate 
    for i in range(1,frameCount):
        for j in range(i+1, i+extraDiag+2):
            minvalue = 10
            minindex = 0
            #This is pretty clumsy, there must be a better way!
            #Argmin is not correct
            for k in range(1, extraDiag+2):
                if(minvalue>errors[i-k, i] + lambdaa*frameAccelCost(i-k, i, j)):
                    minvalue = errors[i-k, i] + lambdaa*frameAccelCost(i-k, i, j)
                    minindex = j
            if(j<=frameCount & i<=frameCount):
                errors[i, j]+= minvalue
                costmatrix [i, j] = minindex       
    
    #Second Pass Trace Back Min Cost
    #Multi-dimensional argmin to set initial values of s and d (basically the min value in the last few rows we can select)
    s = frameCount - 4
    d = s + 1
    
    p = []
    while(s>frameCount):
        #Prepend to List and work our way back up through the cost matrix
        p.insert(0, s)
        b = costmatrix[s, d]
        d = s
        s = b
    return p

#Main Function
#By this unscientific timing the first algo takes 24 seconds and the dictionary based approach is 16 seconds. So roughly 33% quicker
#Repeated with dSSIM we have 7min 6sec and 6min 42 sec. Likely due to the fact that the increase is due to dSSIM being more costly
#int datetime.datetime.utcnow()
#plotError(165, 6)
#print datetime.datetime.utcnow()
#plotErrorDict(165, 6)
print datetime.datetime.utcnow()
arrayPath(165, 6, 4)


2017-07-12 14:34:16.873000
Loading Array
Array Loaded
Speed Up Cost Added


[]